In [144]:
import cv2
import numpy as np
from statistics import mean, mode

In [145]:
def median(arr):
    arr = list(map(int, arr))
    size = len(arr)
    if size % 2:
        return arr[size//2]
    if size != 0:
        return (arr[size//2]+arr[size//2+1])//2
    return 0

In [146]:
def calculateSpatialInfo(img: np.ndarray) -> float:
    """
    Calculate spatial information from a grayscale image using Sobel filters.

    Args:
        img: A grayscale image as a numpy array.

    Returns:
        The spatial information as a float.
    """
    sh = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=1)
    sv = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=1)

    sobel_image = np.sqrt(np.square(sh) + np.square(sv))

    si_mean = np.sum(sobel_image) / (sobel_image.shape[0] * sobel_image.shape[1])
    si_rms = np.sqrt(np.sum(sobel_image ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))
    si_stdev = np.sqrt(np.sum(sobel_image ** 2 - si_mean ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))

    return si_stdev

In [147]:
def getNeighbour(image: np.ndarray, i: int, j: int) -> list:
    neighbour = [[], [], []]
    shape = image.shape
    for di, dj in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
        curr_i = i + di
        curr_j = j + dj
        if curr_i < 0 or curr_i >= shape[0]:
            continue
        if curr_j < 0 or curr_j >= shape[1]:
            continue
        for color in [0, 1, 2]:
            neighbour[color].append(image[curr_i][curr_j][color])
    return neighbour


In [148]:
def polishingImage(resized: np.ndarray, func: callable) -> np.ndarray:
    shape = resized.shape
    polished = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            neighbour = getNeighbour(resized, i, j)
            polished[i][j] = [func(neighbour[0]), func(neighbour[1]), func(neighbour[2])]
    return polished

In [149]:
def resizeImage(original: np.ndarray, x_rate: float, y_rate: float, func: callable = None) -> np.ndarray:
    shape = list(original.shape)
    shape[0] = int(shape[0]*x_rate)
    shape[1] = int(shape[1]*y_rate)
    resized = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            orig_i, orig_j = int(i/x_rate),int(j/y_rate)
            if func:
                neighbour = getNeighbour(original, orig_i, orig_j)
                resized[i][j] = [func(neighbour[0]), func(neighbour[1]), func(neighbour[2])]
            else:
                resized[i][j] = [*original[orig_i][orig_j]]
            
    return resized


In [150]:
prefix = 'images/c'
original = {}
generated = {}

In [151]:
original['normal'] = cv2.imread(f'{prefix}/normal.jpg')
original['big'] = cv2.imread(f'{prefix}/big.jpg')
original['normal_gray'] = cv2.imread(f'{prefix}/normal.jpg', cv2.IMREAD_GRAYSCALE)
original['big_gray'] = cv2.imread(f'{prefix}/big.jpg', cv2.IMREAD_GRAYSCALE)

In [152]:
# Calculate images spatial info
spatial_info = calculateSpatialInfo(original['normal_gray'])
spatial_info


45.468247897791535

In [153]:
# Obtain resized images the was polished based in the original images
generated['big_ob_mode'] = resizeImage(original['normal'], 2, 2, mode)
generated['big_ob_mean'] = resizeImage(original['normal'], 2, 2, mean)
generated['big_ob_median'] = resizeImage(original['normal'], 2, 2, median)

/tmp/ipykernel_25818/2493974170.py:6: RuntimeWarning: overflow encountered in scalar add
  return (arr[size//2]+arr[size//2+1])//2


In [154]:
# Obtain polished images the was polished based in the resized images
generated['big'] = resizeImage(original['normal'], 2, 2)
generated['big_rb_mode'] = polishingImage(generated['big'], mode)
generated['big_rb_mean'] = polishingImage(generated['big'], mean)
generated['big_rb_median'] = polishingImage(generated['big'], median)

In [155]:
for image in generated:
    cv2.imwrite(f'{prefix}/generated/{image}.jpg', generated[image])

In [156]:
results = []
for image in generated:
    generated_gray = cv2.imread(f'{prefix}/generated/{image}.jpg', cv2.IMREAD_GRAYSCALE)
    results.append((cv2.PSNR(original['big_gray'], generated_gray), image))

In [158]:
results.sort(reverse=True)
for psnr,name in results:
    print(f'{name:13s}: PSNR {psnr:.4f}')

big_rb_mean  : PSNR 22.5763
big_rb_mode  : PSNR 22.4182
big          : PSNR 22.3938
big_rb_median: PSNR 21.2658
big_ob_mean  : PSNR 19.7491
big_ob_mode  : PSNR 18.7909
big_ob_median: PSNR 6.3367
